In [1]:
import pandas as pd


In [7]:
data=pd.read_csv('/content/PS_20174392719_1491204439457_log.csv')

/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/c_parser_wrapper.py:373: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(


In [3]:
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0


In [5]:
data.shape

(246948, 11)

In [8]:
import numpy as np
import pandas as pd
import gym
data = data.drop(['step', 'type', 'nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1)
data = (data - data.mean()) / data.std()  # Normalize numerical features

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
data.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
       'newbalanceDest', 'isFraud'],
      dtype='object')

In [14]:
class FraudDetectionEnv(gym.Env):
    def __init__(self, data):
        super(FraudDetectionEnv, self).__init__()
        self.data = data
        self.state = None
        self.total_transactions = len(data)
        self.current_transaction_index = 0
        self.action_space = gym.spaces.Discrete(2)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(len(data.columns),))
        self.reward_range = (-1, 1)

    def reset(self):
        self.current_transaction_index = 0
        self.state = self.data.iloc[self.current_transaction_index].values
        return self._next_observation()

    def _next_observation(self):
        return self.state

    def step(self, action):
        done = (self.current_transaction_index == self.total_transactions - 1)
        next_state = self.data.iloc[self.current_transaction_index].values
        reward = self._get_reward(action)
        self.current_transaction_index += 1
        self.state = next_state
        return next_state, reward, done, {}

    def _get_reward(self, action):
        return -1 if action == 0 else 1

env = FraudDetectionEnv(data)
state_bins = [np.linspace(-5, 5, 10) for _ in range(len(data.columns))]
q_table = np.zeros((10,) * len(data.columns) + (env.action_space.n,))
learning_rate = 0.1
discount_factor = 0.99
epsilon = 0.1
num_episodes = 10
def discretize_state(state):
    discretized_state = []
    for s, bins in zip(state, state_bins):
        if s <= bins[0]:
            discretized_state.append(0)
        elif s >= bins[-1]:
            discretized_state.append(len(bins) - 1)
        else:
            discretized_state.append(np.digitize(s, bins) - 1)
    return tuple(discretized_state)

for episode in range(num_episodes):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        if np.random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[tuple(discretize_state(state))])
        next_state, reward, done, _ = env.step(action)
        q_table[tuple(discretize_state(state)) + (action,)] += learning_rate * (reward + discount_factor * np.max(q_table[tuple(discretize_state(next_state))]) - q_table[tuple(discretize_state(state)) + (action,)])
        total_reward += reward
        state = next_state

    print(f"Episode {episode + 1}, Total Reward: {total_reward}")

def evaluate_agent(env, q_table, num_episodes):
    total_rewards = []
    for _ in range(num_episodes):
        state = env.reset()
        done = False
        episode_reward = 0

        while not done:
            action = np.argmax(q_table[tuple(discretize_state(state))])
            next_state, reward, done, _ = env.step(action)
            episode_reward += reward
            state = next_state

        total_rewards.append(episode_reward)

    avg_reward = np.mean(total_rewards)
    print(f"Average Reward per Episode: {avg_reward}")

evaluate_agent(env, q_table, num_episodes)

Episode 1, Total Reward: 174322
Episode 2, Total Reward: 183972
Episode 3, Total Reward: 188326
Episode 4, Total Reward: 193514
Episode 5, Total Reward: 195040
Episode 6, Total Reward: 202064
Episode 7, Total Reward: 202018
Episode 8, Total Reward: 203610
Episode 9, Total Reward: 204178
Episode 10, Total Reward: 206956
Episode 11, Total Reward: 207380
Episode 12, Total Reward: 208896
Episode 13, Total Reward: 210834
Episode 14, Total Reward: 211864
Episode 15, Total Reward: 212954
Episode 16, Total Reward: 213860
Episode 17, Total Reward: 214370
Episode 18, Total Reward: 214654
Episode 19, Total Reward: 215538
Episode 20, Total Reward: 216212
Average Reward per Episode: 198390.0


In [32]:
def discretize_state(state):
    discretized_state = []
    for s, bins in zip(state, state_bins):
        if s <= bins[0]:
            discretized_state.append(0)
        elif s >= bins[-1]:
            discretized_state.append(len(bins) - 1)
        else:
            discretized_state.append(np.digitize(s, bins) - 1)
    return tuple(discretized_state)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246948 entries, 0 to 246947
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   amount          246948 non-null  float64
 1   oldbalanceOrg   246948 non-null  float64
 2   newbalanceOrig  246947 non-null  float64
 3   oldbalanceDest  246947 non-null  float64
 4   newbalanceDest  246947 non-null  float64
 5   isFraud         246947 non-null  float64
dtypes: float64(6)
memory usage: 11.3 MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
mean_val_feature1=data['amount'].mean()
mean_val_feature2=data['oldbalanceOrg'].mean()
mean_val_feature3=data['newbalanceOrig'].mean()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
std_val_feature1=data['amount'].std()
std_val_feature2=data['oldbalanceOrg'].std()
std_val_feature3=data['newbalanceOrig'].std()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
import numpy as np
import joblib
q_table=joblib.load('/content/q_table.pkl')
df=pd.read_csv('/content/PS_20174392719_1491204439457_log.csv')
custom_sample_data = [181.0, 181.0, 0.0, 0.0, 0.0]  # Example sample data
mean_values = [df['amount'].mean(),df['oldbalanceOrg'].mean(),df['newbalanceOrig'].mean(),df['oldbalanceDest'].mean(),df['newbalanceDest'].mean()]  # Fill in with mean values used during training
std_values =  [df['amount'].std(),df['oldbalanceOrg'].std(),df['newbalanceOrig'].std(),df['oldbalanceDest'].std(),df['newbalanceDest'].std()]
normalized_features = [(feature - mean_val) / std_val for feature, mean_val, std_val in zip(custom_sample_data, mean_values, std_values)]
discretized_state = discretize_state(normalized_features)
action = np.argmax(q_table[tuple(discretized_state)])
action_str = "Approve" if action == 0 else "Decline"
print("Action chosen for custom sample data:", action_str)


Action chosen for custom sample data: Decline


/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/c_parser_wrapper.py:373: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(


In [20]:
import joblib
joblib.dump(q_table, 'q_table.pkl')


['q_table.pkl']

In [24]:
data.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,-0.535675,-0.253213,-0.259799,-0.404388,-0.455316,-0.025542
1,-0.560866,-0.305972,-0.309078,-0.404388,-0.455316,-0.025542
2,-0.566183,-0.313437,-0.315857,-0.404388,-0.455316,39.151323
3,-0.566183,-0.313437,-0.315857,-0.395448,-0.455316,39.151323
4,-0.529900,-0.298777,-0.305405,-0.404388,-0.455316,-0.025542
5,-0.542062,-0.294416,-0.299755,-0.404388,-0.455316,-0.025542
6,-0.544304,-0.248586,-0.254277,-0.404388,-0.455316,-0.025542
7,-0.541923,-0.251104,-0.257026,-0.404388,-0.455316,-0.025542
8,-0.554044,-0.312555,-0.315857,-0.404388,-0.455316,-0.025542
9,-0.549895,-0.298718,-0.303134,-0.386705,-0.439926,-0.025542


In [26]:
data.columns

Index(['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
       'newbalanceDest', 'isFraud'],
      dtype='object')

In [27]:
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0


In [28]:
data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,-0.535675,-0.253213,-0.259799,-0.404388,-0.455316,-0.025542
1,-0.560866,-0.305972,-0.309078,-0.404388,-0.455316,-0.025542
2,-0.566183,-0.313437,-0.315857,-0.404388,-0.455316,39.151323
3,-0.566183,-0.313437,-0.315857,-0.395448,-0.455316,39.151323
4,-0.529900,-0.298777,-0.305405,-0.404388,-0.455316,-0.025542
...,...,...,...,...,...,...
246943,-0.554099,-0.313501,-0.315857,-0.404388,-0.455316,-0.025542
246944,-0.527768,-0.313501,-0.315857,-0.404388,-0.455316,-0.025542
246945,-0.526202,-0.313501,-0.315857,-0.404388,-0.455316,-0.025542
246946,-0.562578,-0.313501,-0.315857,-0.404388,-0.455316,-0.025542
